In [20]:
import re
import chromadb

# Connexion à ta base Chroma
client = chromadb.PersistentClient(path="C:\\Users\\SCHENNOUFI\\OneDrive - Micropole\\Bureau\\WorkSpace\\IA GEN\\CHATBOT\\prod\\chromadb copy")

In [17]:
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
#from back import connection
#from back.connection import create_llm_chat_langchain, create_llm_langchain, create_embeddings_azureopenai, create_openai_native
import os
from langchain_openai import ChatOpenAI, OpenAI, OpenAIEmbeddings
AZURE_AOAI_API_VERSION = "2024-08-01-preview"
AZURE_AOAI_MODEL_GPT3_TURBO = "gpt35turbo"
AZURE_AOAI_MODEL_GPT4O = "gpt-4o"
AZURE_AOAI_MODEL_GPT4OMINI = "gpt-4o-mini"
AZURE_EMBEDDING_MODEL = "text-embedding-ada-002"
# Load .env
load_dotenv(find_dotenv())

# === Now you can use it exactly like OpenAI SDK ===
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def create_embeddings_azureopenai(model):

    embeddings = OpenAIEmbeddings(
        openai_api_key = os.getenv("OPENAI_API_KEY"),
        model=model
    )

    return embeddings


# Create embeddings
embeddings = create_embeddings_azureopenai(AZURE_EMBEDDING_MODEL)

In [8]:
collections = client.list_collections()
for c in collections:
    print(c.name)

langchain


In [22]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name=AZURE_EMBEDDING_MODEL)

In [23]:
collection = client.get_collection("langchain", embedding_function=embedding_function)

In [11]:
results = collection.get(include=["documents", "metadatas"]) 
EXCLUDED_NUMBERS = {2022, 2023, 2024}

In [24]:
EXCLUDED_NUMBERS = {2022, 2023, 2024}

updated_metas = []
for metadata in results["metadatas"]:
    new_meta = {}
    for k, v in metadata.items():
        if isinstance(v, (int, float)) and v not in EXCLUDED_NUMBERS:
            new_meta[k] = round(v * 0.3, 2)
        else:
            new_meta[k] = v
    updated_metas.append(new_meta)

def scale_number(match):
    num_str = match.group()
    if num_str in EXCLUDED_NUMBERS:
        return num_str
    return str(round(float(num_str) * 0.3, 2))
# Mise à jour des documents
# On utilise une expression régulière pour trouver les nombres dans les documents

updated_docs = []
for doc in results["documents"]:
    updated_doc = re.sub(r"\b\d+(\.\d+)?\b", scale_number, doc)
    updated_docs.append(updated_doc)

# Re-upsert avec documents + métadonnées
for i in range(len(results["ids"])):
    doc_id = results["ids"][i]
    doc = updated_docs[i]
    meta = updated_metas[i]

    # On met à jour le document (obligatoire) et les métadonnées (optionnelles)
    collection.upsert(
        ids=[doc_id],
        documents=[doc],
        metadatas=[meta]
    )


In [26]:
results_after = collection.get(include=["documents", "metadatas"])

for i in range(len(results["ids"])):
    old_doc = results["documents"][i]
    new_doc = results_after["documents"][i]
    doc_id = results["ids"][i]

    if old_doc != new_doc:
        print(f"✅ Différence détectée pour le document ID: {doc_id}")
        print("Ancien :", old_doc)
        print("Nouveau:", new_doc)
        print("-" * 50)
    else:
        print(f"🔸 Aucun changement pour le document ID: {doc_id}")


✅ Différence détectée pour le document ID: 63a6918c-31e7-4be5-82c8-0927c8b57c63
Ancien : 2024 FY Total Savings: 1,7M€
2024 FY P&L Improvement: 1,1M€
2024 FY Savings On Project: 51,6K€
2024 FY Cost Avoidance: 524,6K€
% Worked Spend Baseline 2024 FY over P&L Improvement: 5%
Variation % 2023 FY vs 2024 FY P&L Improvement: (Vide)
2024 FY P&L Improv. vs Savings on Project vs Cost Avoid. by Family
2024 FY Savings by Type per Year
2024 FY Top 5 Actions of P&L Improvement Savings
P&L Savings renouvellement 2024 NETHEOS Floa: 0,49M€
Professional Services P&L following RFA for FLOA: 0,38M€
Savings P&L Azure FLOA Bank: 0,20M€
[Savings P&L] Renew support Basikon FLOA: 0,06M€
P&L - Renew license Tableau (Salesforce) - Multi-entities: 0,00M€

Family | Value
Technology | 0,05M€
Technology | 0,22M€
Banking Services | 0,27M€
Corporate Services | 0,03M€
Professional services | 0,00M€

Action | Value
P&L Savings renouvellement 2024 NETHEOS Floa | 0,49M€
Professional Services P&L following RFA for FLOA | 

In [34]:
client.delete_collection(name="langchain")

In [35]:
collection = client.get_or_create_collection(
    name="langchain",
    embedding_function=embedding_function
)

In [36]:
collection.add(
    ids=results["ids"],
    documents=updated_docs,
    metadatas=updated_metas
)


In [37]:
collections = client.list_collections()

print("📦 Collections actuellement disponibles :")
for col in collections:
    print("-", col.name)


📦 Collections actuellement disponibles :
- langchain


In [ ]:
# api.py
from fastapi import FastAPI
from pydantic import BaseModel
import joblib

# Définition du schéma d’entrée
class Features(BaseModel):
    age: float
    salary: float
    # ajoutez autant de champs que nécessaire

# Chargement du modèle
model = joblib.load("model.pkl")

app = FastAPI(
    title="API de Prédiction",
    description="API pour prédire des valeurs à partir de caractéristiques d'entrée",
    version="1.0.0",
    contact={
        "name": "Votre Nom"}
)

@app.post("/predict/")
def predict(data: Features):
    # Conversion en tableau 2D pour scikit-learn
    X = [[data.age, data.salary]]
    pred = model.predict(X)
    return {"prediction": pred.tolist()}

# Pour lancer : uvicorn api:app --reload


In [39]:
import uvicorn
# Pour lancer le serveur FastAPI
if __name__ == "__main__":
    uvicorn.run("api:app", host="0.0.0", port=8000, reload=True)


INFO:     Will watch for changes in these directories: ['c:\\Users\\SCHENNOUFI\\OneDrive - Micropole\\Bureau\\WorkSpace\\IA GEN\\CHATBOT\\prod\\Learning']
ERROR:    [Errno 11001] getaddrinfo failed


SystemExit: 1

C:\Users\SCHENNOUFI\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3680: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
